In [37]:
# import des librairies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
# clean text data
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

# # prep the data for the model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Concatenate


# model
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Concatenate

ImportError: Traceback (most recent call last):
  File "C:\Users\selmane.kenzari\AppData\Local\anaconda3\envs\jobmarket\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 62, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: Module use of python38.dll conflicts with this version of Python.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [8]:
# import des données
jobs_df = pd.read_csv("../data/silver/silver_data.csv")
jobs_salary = pd.read_csv("../data/job_salary.csv")
company_salary = pd.read_csv("../data/company_salary.csv")
jobs_df.head()

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,ANALYSTE APPLICATIF DATA FINANCE RH (H/F),Dans le cadre d'un important projet avec un gr...,92 - ISSY LES MOULINEAUX,Actualisé le 07 août 2023,35H Travail en journée,50000.0 - 60000.0,Contrat à durée indéterminée,2 ans,NaN,Adapter les outils de traitement statistique d...,Cadre,Conseil en systèmes et logiciels informatiques,INFORMATIS T.S.,6 à 9 salariés,NaN,NaN,92,ISSY LES MOULINEAUX,2023/08/07
1,Ingénieur DATA (H/F),Nous recherchons : un Ingénieur DATA H/F.\n\nP...,92 - PUTEAUX,Actualisé le 07 août 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,2 ans,"Bac+3, Bac+4 ou équivalents","Application web,Concevoir une application web,...",Agent de maîtrise,Conseil pour les affaires et autres conseils d...,IL CONSULTING,NaN,NaN,"gitlab, kubernetes, mongodb",92,PUTEAUX,2023/08/07
2,Data Integrity Analyst (H/F),ADECCO Lyon Pharma recherche un Data Integrity...,69 - ST PRIEST,Actualisé le 07 août 2023,35H Travail en journée,2311.0,Mission intérimaire - 12 Mois,Débutant accepté,NaN,Adapter les outils de traitement statistique d...,Employé qualifié,Activités des agences de travail temporaire,ADECCO Onsite,0 salarié (n'ayant pas d'effectif au 31/12 mai...,"Premier réseau d'agences d'emploi en France, A...",NaN,69,ST PRIEST,2023/08/07
3,Technicien Data (h/f),"Adecco PONTARLIER, recherche pour l'un de ses ...",25 - PONTARLIER,Actualisé le 07 août 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,Débutant accepté,NaN,"Concevoir et gérer un projet,Concevoir un logi...",Employé qualifié,Activités des agences de travail temporaire,ADECCO,6 à 9 salariés,"Premier réseau d'agences d'emploi en France, A...","tableau, power bi",25,PONTARLIER,2023/08/07
4,Data Engineer (H/F),"Selon vos expériences, votre mission sera de :...",33 - BORDEAUX,Actualisé le 07 août 2023,38H30 Travail en journée,30000.0 - 50000.0,Contrat à durée indéterminée,3 ans,NaN,NaN,Cadre,Conseil en systèmes et logiciels informatiques,THELIO,20 à 49 salariés,NaN,"tableau, power bi, talend, databricks, snowfla...",33,BORDEAUX,2023/08/07


In [9]:
jobs_salary.head()

,Job,salary($),salary_type,Base Salary,Bonus,Total Pay,Profit Sharing,Commission,Hourly Rate
0,ABAP Developer,90000.0,year,$68k - $120k,$2k - $20k,$67k - $132k,NaN,NaN,NaN
1,ASIC Design Engineer,108330.0,year,$83k - $148k,$1k - $15k,$85k - $164k,$2k - $15k,NaN,NaN
2,ASIC Engineer,116546.0,year,$59k - $165k,$503 - $22k,$59k - $179k,NaN,NaN,NaN
3,ASP.NET Developer,71152.0,year,$50k - $94k,$993 - $10k,$47k - $96k,$2k - $5k,NaN,NaN
4,Actuarial Associate,82113.0,year,$65k - $101k,$2k - $13k,$67k - $109k,$2k - $7k,NaN,NaN


In [10]:
company_salary.head()

,company,avg_salary($),salary_type
0,A.B._Data%2C_Ltd.,NaN,year
1,A-Tech,20.0,hour
2,A_Wireless,NaN,year
3,A_%26_A_Telecom,83.0,year
4,A_%26_T_Systems,NaN,year


## Nettoyage et préparation des données

In [22]:
# préparer un df avec uniquement les colonnes à utiliser pour le modèle
df = jobs_df[["jobs", "description", "ID_dep", "ville", "date", "experience", "skills", "tools", "industry", "company", "company_description"]]
df.head()

,jobs,description,ID_dep,ville,date,experience,skills,tools,industry,company,company_description
0,ANALYSTE APPLICATIF DATA FINANCE RH (H/F),Dans le cadre d'un important projet avec un gr...,92,ISSY LES MOULINEAUX,2023/08/07,2 ans,Adapter les outils de traitement statistique d...,NaN,Conseil en systèmes et logiciels informatiques,INFORMATIS T.S.,NaN
1,Ingénieur DATA (H/F),Nous recherchons : un Ingénieur DATA H/F.\n\nP...,92,PUTEAUX,2023/08/07,2 ans,"Application web,Concevoir une application web,...","gitlab, kubernetes, mongodb",Conseil pour les affaires et autres conseils d...,IL CONSULTING,NaN
2,Data Integrity Analyst (H/F),ADECCO Lyon Pharma recherche un Data Integrity...,69,ST PRIEST,2023/08/07,Débutant accepté,Adapter les outils de traitement statistique d...,NaN,Activités des agences de travail temporaire,ADECCO Onsite,"Premier réseau d'agences d'emploi en France, A..."
3,Technicien Data (h/f),"Adecco PONTARLIER, recherche pour l'un de ses ...",25,PONTARLIER,2023/08/07,Débutant accepté,"Concevoir et gérer un projet,Concevoir un logi...","tableau, power bi",Activités des agences de travail temporaire,ADECCO,"Premier réseau d'agences d'emploi en France, A..."
4,Data Engineer (H/F),"Selon vos expériences, votre mission sera de :...",33,BORDEAUX,2023/08/07,3 ans,NaN,"tableau, power bi, talend, databricks, snowfla...",Conseil en systèmes et logiciels informatiques,THELIO,NaN


In [25]:
df["jobs"] = df["jobs"].str.replace(" h/f", "")
df["jobs"] = df["jobs"].str.replace("\s*\([hH]/[fF]\)\s*", " ").str.strip()
df["jobs"] = df["jobs"].str.replace(" (H/F)", "")
df["jobs"] = df["jobs"].str.replace(" H/F", "")
df["jobs"] = df["jobs"].str.replace(" F/H", "")
df["jobs"] = df["jobs"].str.replace(" (F/H)", "")
df["jobs"] = df["jobs"].str.replace(" \(\)", "")

C:\Users\selmane.kenzari\AppData\Local\Temp\ipykernel_19336\3925056316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["jobs"] = df["jobs"].str.replace(" h/f", "")
C:\Users\selmane.kenzari\AppData\Local\Temp\ipykernel_19336\3925056316.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["jobs"] = df["jobs"].str.replace("\s*\([hH]/[fF]\)\s*", " ").str.strip()
C:\Users\selmane.kenzari\AppData\Local\Temp\ipykernel_19336\3925056316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [26]:
df.dtypes

jobs                   object
description            object
ID_dep                 object
ville                  object
date                   object
experience             object
skills                 object
tools                  object
industry               object
company                object
company_description    object
dtype: object

In [27]:
# Prétraitement des données
def preprocess_text(text):
    # Suppression des caractères spéciaux et de la ponctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Conversion en minuscules
    text = text.lower()
    
    # Suppression des mots vides
    stop_words = set(stopwords.words('french'))  
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Rejoindre les tokens prétraités en une seule chaîne
    processed_text = ' '.join(tokens)
    
    return processed_text

In [28]:
df["description"][0]

"Dans le cadre d'un important projet avec un groupe international, nous recherchons un Analyste applicatif data domaine finance et rh.\nIntégré à la dsi et encadré par son responsable, vous pilotez et contribué au maintien de la qualité de fonctionnement et au support des applications placées sous votre responsabilité.\n\nVous êtes féru de manipulation de données et possédez une première expérience dans le domaine IT lié à la Finance et ou RH\n\nDe formations supérieure en informatique, vous avez une expérience de 2 ans minimum dans un poste équivalent. Vous possédez une grande aisance dans la manipulation de données et souhaitez travailler dans des projets d'envergure dans un contexte international.\nAnglais professionnel nécessaire, contexte international."

In [29]:
df["description"] = df["description"].apply(preprocess_text)
df["company_description"] = df["company_description"].astype(str)
df["company_description"] = df["company_description"].apply(preprocess_text)
df["experience"] = df["experience"].apply(preprocess_text)

C:\Users\selmane.kenzari\AppData\Local\Temp\ipykernel_19336\3973741798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["description"] = df["description"].apply(preprocess_text)
C:\Users\selmane.kenzari\AppData\Local\Temp\ipykernel_19336\3973741798.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["company_description"] = df["company_description"].astype(str)
C:\Users\selmane.kenzari\AppData\Local\Temp\ipykernel_19336\3973741798.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [30]:
df["experience"][1]

'2 an'

In [31]:
df.head()

,jobs,description,ID_dep,ville,date,experience,skills,tools,industry,company,company_description
0,ANALYSTE APPLICATIF DATA FINANCE RH,cadre dun important projet groupe internationa...,92,ISSY LES MOULINEAUX,2023/08/07,2 an,Adapter les outils de traitement statistique d...,NaN,Conseil en systèmes et logiciels informatiques,INFORMATIS T.S.,nan
1,Ingénieur DATA,recherchons ingénieur data hf compte direction...,92,PUTEAUX,2023/08/07,2 an,"Application web,Concevoir une application web,...","gitlab, kubernetes, mongodb",Conseil pour les affaires et autres conseils d...,IL CONSULTING,nan
2,Data Integrity Analyst,adecco lyon pharma recherche data integrity an...,69,ST PRIEST,2023/08/07,débutant accepté,Adapter les outils de traitement statistique d...,NaN,Activités des agences de travail temporaire,ADECCO Onsite,premier réseau dagences demploi france adecco ...
3,Technicien Data,adecco pontarlier recherche lun client importa...,25,PONTARLIER,2023/08/07,débutant accepté,"Concevoir et gérer un projet,Concevoir un logi...","tableau, power bi",Activités des agences de travail temporaire,ADECCO,premier réseau dagences demploi france adecco ...
4,Data Engineer,selon expériences mission définir architecture...,33,BORDEAUX,2023/08/07,3 an,NaN,"tableau, power bi, talend, databricks, snowfla...",Conseil en systèmes et logiciels informatiques,THELIO,nan


In [32]:
df.shape

(1000, 11)

In [34]:
df.isnull().sum()

jobs                     0
description              0
ID_dep                   0
ville                  123
date                     0
experience               0
skills                 848
tools                  409
industry               422
company                431
company_description      0
dtype: int64

## Baseline Model

In [ ]:
# Combine all your text data for effective tokenization
all_text = skills + location + contract_type + job_description + job_title

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_text)

# Convert text data to sequences
skills_seq = tokenizer.texts_to_sequences(skills)
location_seq = tokenizer.texts_to_sequences(location)
contract_seq = tokenizer.texts_to_sequences(contract_type)
description_seq = tokenizer.texts_to_sequences(job_description)
title_seq = tokenizer.texts_to_sequences(job_title)

In [ ]:
MAXLEN = 300  # You can adjust this based on your data

skills_seq_padded = pad_sequences(skills_seq, maxlen=MAXLEN)
location_seq_padded = pad_sequences(location_seq, maxlen=MAXLEN)
contract_seq_padded = pad_sequences(contract_seq, maxlen=MAXLEN)
description_seq_padded = pad_sequences(description_seq, maxlen=MAXLEN)
title_seq_padded = pad_sequences(title_seq, maxlen=MAXLEN)

In [ ]:
EMBEDDING_DIM = 100  # or the size of your embeddings if you're using pre-trained
VOCAB_SIZE = len(tokenizer.word_index) + 1  # +1 for padding

# Define an input sequence and process it
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(encoder_inputs)

# You might want separate encoders for each input feature or one concatenated feature
# For simplicity, I'll use one encoder for demonstration
encoder_lstm = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Set up the decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(decoder_inputs)
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(VOCAB_SIZE, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & train
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([skills_seq_padded, title_seq_padded], description_seq_padded, batch_size=64, epochs=10)